In [62]:
import pandas as pd
import numpy as np

In [146]:
# Read all the 6 indicators (CSVs)
df_economic = pd.read_csv("../data/resources/readiness/melted_economic.csv")

df_governance = pd.read_csv("../data/resources/readiness/melted_governance.csv")
del df_governance["Name"]
df_social = pd.read_csv("../data/resources/readiness/melted_social.csv")
del df_social["Name"]

df_ecosystems = pd.read_csv("../data/resources/vulnerability/melted_ecosystems.csv")
del df_ecosystems["Name"]
df_habitat = pd.read_csv("../data/resources/vulnerability/melted_habitat.csv")
del df_habitat["Name"]
df_infrastructure = pd.read_csv("../data/resources/vulnerability/melted_infrastructure.csv")
del df_infrastructure["Name"]

# Also read political stability csv
df_politicalStability = pd.read_csv("../data/resources/indicators/id_gove_01/melted_governance.csv")
del df_politicalStability["Name"]

In [147]:
df_indicators = pd.merge(df_economic, df_governance, how="outer", on=["ISO3", "Year"])
df_indicators = pd.merge(df_indicators, df_social, how="outer", on=["ISO3", "Year"])
df_indicators = pd.merge(df_indicators, df_ecosystems, how="outer", on=["ISO3", "Year"])
df_indicators = pd.merge(df_indicators, df_habitat, how="outer", on=["ISO3", "Year"])
df_indicators = pd.merge(df_indicators, df_infrastructure, how="outer", on=["ISO3", "Year"])

In [148]:
df_indicators = pd.merge(df_indicators, df_politicalStability, how="outer", on=["ISO3", "Year"])
df_indicators

,ISO3,Name,Year,value_economic,no_value_economic,value_governance,no_value_governance,value_social,no_value_social,value_ecosystems,no_value_ecosystems,value_habitat,no_value_habitat,value_infrastructure,no_value_infrastructure,value_governance_isolated,no_value_governance_isolated
0,AFG,Afghanistan,1995,0.496497,False,0.138771,False,0.295982,False,0.516028,False,0.603153,False,0.383710,True,0.154928,False
1,ALB,Albania,1995,0.393305,False,0.383518,False,0.225699,False,0.484339,False,0.509650,False,0.474276,False,0.512667,False
2,DZA,Algeria,1995,0.416625,False,0.292892,False,0.194779,False,0.411816,False,0.467930,False,0.157379,False,0.263959,False
3,AND,Andorra,1995,0.410408,True,0.501894,True,0.156764,False,0.477737,True,0.527740,True,0.383710,True,0.578457,True
4,AGO,Angola,1995,0.289057,False,0.201718,False,0.083156,False,0.544357,False,0.648126,False,0.326570,False,0.216279,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4987,VEN,"Venezuela, Bolivarian Republic o",2020,0.111244,False,0.127607,False,0.331404,False,0.351461,False,0.604549,False,0.208577,False,0.309044,False
4988,VNM,Viet Nam,2020,0.490740,False,0.468500,False,0.316170,False,0.555078,False,0.433713,False,0.562195,False,0.557445,False
4989,YEM,Yemen,2020,0.379586,False,0.120174,False,0.243894,False,0.570795,False,0.611543,False,0.340751,False,0.110621,False
4990,ZMB,Zambia,2020,0.413829,False,0.399141,False,0.150245,False,0.433906,False,0.594931,False,0.560437,False,0.548028,False


In [150]:
# To armonize data, let's change "United states" as "USA"
invalid_rows = df_indicators["Name"] == "United States"
df_indicators.loc[invalid_rows, "Name"] = "USA"

In [151]:
# Create geometric mean for these 6 indicators (EGSEHI)
df_indicators["EGSEHI"] = df_indicators["value_economic"] * df_indicators["value_governance"] * df_indicators["value_social"] * \
                            df_indicators["value_ecosystems"] * df_indicators["value_habitat"] * df_indicators["value_infrastructure"]

root = 6
df_indicators[f"EGSEHI_{root}root"] = np.power(df_indicators["EGSEHI"], root)

In [152]:
df_indicators["Name"] = df_indicators["Name"].str.upper()

In [153]:
df_indicators.to_csv("../data/indicators_conformed_file.csv", index=False)

In [154]:
df_indicators

,ISO3,Name,Year,value_economic,no_value_economic,value_governance,no_value_governance,value_social,no_value_social,value_ecosystems,no_value_ecosystems,value_habitat,no_value_habitat,value_infrastructure,no_value_infrastructure,value_governance_isolated,no_value_governance_isolated,EGSEHI,EGSEHI_6root
0,AFG,AFGHANISTAN,1995,0.496497,False,0.138771,False,0.295982,False,0.516028,False,0.603153,False,0.383710,True,0.154928,False,0.002435,2.086886e-16
1,ALB,ALBANIA,1995,0.393305,False,0.383518,False,0.225699,False,0.484339,False,0.509650,False,0.474276,False,0.512667,False,0.003986,4.008480e-15
2,DZA,ALGERIA,1995,0.416625,False,0.292892,False,0.194779,False,0.411816,False,0.467930,False,0.157379,False,0.263959,False,0.000721,1.402677e-19
3,AND,ANDORRA,1995,0.410408,True,0.501894,True,0.156764,False,0.477737,True,0.527740,True,0.383710,True,0.578457,True,0.003124,9.292030e-16
4,AGO,ANGOLA,1995,0.289057,False,0.201718,False,0.083156,False,0.544357,False,0.648126,False,0.326570,False,0.216279,False,0.000559,3.039842e-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4987,VEN,"VENEZUELA, BOLIVARIAN REPUBLIC O",2020,0.111244,False,0.127607,False,0.331404,False,0.351461,False,0.604549,False,0.208577,False,0.309044,False,0.000208,8.213285e-23
4988,VNM,VIET NAM,2020,0.490740,False,0.468500,False,0.316170,False,0.555078,False,0.433713,False,0.562195,False,0.557445,False,0.009838,9.068718e-13
4989,YEM,YEMEN,2020,0.379586,False,0.120174,False,0.243894,False,0.570795,False,0.611543,False,0.340751,False,0.110621,False,0.001323,5.370178e-18
4990,ZMB,ZAMBIA,2020,0.413829,False,0.399141,False,0.150245,False,0.433906,False,0.594931,False,0.560437,False,0.548028,False,0.003590,2.142007e-15


## Production

In [155]:
df_production = pd.read_csv("../data/Production_one_file.csv")
df_production["Year"] = pd.to_numeric(df_production["Year"], downcast="integer")
df_production["Country"] = df_production["Country"].str.upper()
df_production["Product"] = df_production["Product"].str.upper()
df_production.rename(columns={"Value": "Domestic Production Value (P_AC)"}, inplace=True)

In [156]:
df_production

,Year,Country,Domestic Production Value (P_AC),Product
0,1970,ALBANIA,NaN,ALUMINIUM
1,1971,ALBANIA,NaN,ALUMINIUM
2,1972,ALBANIA,NaN,ALUMINIUM
3,1973,ALBANIA,NaN,ALUMINIUM
4,1974,ALBANIA,NaN,ALUMINIUM
...,...,...,...,...
69530,2016,RUSSIAN FEDERATION,5.584791e+08,PETROLEUM
69531,2017,RUSSIAN FEDERATION,5.585467e+08,PETROLEUM
69532,2018,RUSSIAN FEDERATION,5.678849e+08,PETROLEUM
69533,2019,RUSSIAN FEDERATION,5.733888e+08,PETROLEUM


## Join Production and EGSEHI

In [157]:
df_production_egsehi = pd.merge(df_production, df_indicators, how="left", left_on=["Year", "Country"], right_on=["Year", "Name"])
del df_production_egsehi["Name"]

In [158]:
df_production_egsehi

,Year,Country,Domestic Production Value (P_AC),Product,ISO3,value_economic,no_value_economic,value_governance,no_value_governance,value_social,...,value_ecosystems,no_value_ecosystems,value_habitat,no_value_habitat,value_infrastructure,no_value_infrastructure,value_governance_isolated,no_value_governance_isolated,EGSEHI,EGSEHI_6root
0,1970,ALBANIA,NaN,ALUMINIUM,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1971,ALBANIA,NaN,ALUMINIUM,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1972,ALBANIA,NaN,ALUMINIUM,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1973,ALBANIA,NaN,ALUMINIUM,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1974,ALBANIA,NaN,ALUMINIUM,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69530,2016,RUSSIAN FEDERATION,5.584791e+08,PETROLEUM,RUS,0.624176,False,0.358850,False,0.608937,...,0.414390,False,0.517701,False,0.158387,False,0.407352,False,0.004634,9.908522e-15
69531,2017,RUSSIAN FEDERATION,5.585467e+08,PETROLEUM,RUS,0.642995,False,0.365093,False,0.582633,...,0.413218,False,0.513469,False,0.158387,False,0.459903,False,0.004596,9.430271e-15
69532,2018,RUSSIAN FEDERATION,5.678849e+08,PETROLEUM,RUS,0.645651,False,0.370905,False,0.607808,...,0.412046,False,0.509131,False,0.158387,False,0.477004,False,0.004836,1.279774e-14
69533,2019,RUSSIAN FEDERATION,5.733888e+08,PETROLEUM,RUS,0.670757,False,0.378865,False,0.599214,...,0.413524,False,0.512174,False,0.158387,False,0.474219,False,0.005108,1.776735e-14


### Join HHI index table with production JOIN EGSEHI table

In [159]:
df_hhi_master = pd.read_csv("../data/hhi_data/df_hhi_master.csv")
df_hhi_master

c:\Users\zacat\OneDrive\Documentos\Investigacion\investigacion-gnn-cic\rb_venv3715\lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (1,5,27,31,40,44,46,48,53,54) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Year,Country,Value,Product,reporterCode,reporterDesc_x,Total_value_YearProduct,Share in % (production),Share HHI Production,HHI_production,...,cifvalue,fobvalue,primaryValue,legacyEstimationFlag,isReported,isAggregate,Total_netWgt_YearProduct,Share in % (exports),Share HHI Exports,HHI_exports
0,1970,ALBANIA,NaN,ALUMINIUM,8.0,ALBANIA,57280000.0,NaN,NaN,1026.755380,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1971,ALBANIA,NaN,ALUMINIUM,8.0,ALBANIA,60982000.0,NaN,NaN,1132.922206,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1972,ALBANIA,NaN,ALUMINIUM,8.0,ALBANIA,68897000.0,NaN,NaN,1128.647391,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1973,ALBANIA,NaN,ALUMINIUM,8.0,ALBANIA,75087000.0,NaN,NaN,1185.642408,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1974,ALBANIA,NaN,ALUMINIUM,8.0,ALBANIA,83974000.0,NaN,NaN,1223.009237,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87512,2020,NaN,NaN,ZIRCONIUM,757.0,NaN,NaN,NaN,NaN,NaN,...,0.0,266.514,266.514,6.0,False,True,5.196047e+08,0.000005,2.314909e-11,1210.445136
87513,2020,NaN,NaN,ZIRCONIUM,784.0,NaN,NaN,NaN,NaN,NaN,...,0.0,1421075.128,1421075.128,0.0,False,True,5.196047e+08,0.159325,2.538457e-02,1210.445136
87514,2020,NaN,NaN,ZIRCONIUM,792.0,NaN,NaN,NaN,NaN,NaN,...,0.0,161170.000,161170.000,0.0,False,True,5.196047e+08,0.017397,3.026514e-04,1210.445136
87515,2020,NaN,NaN,ZIRCONIUM,826.0,NaN,NaN,NaN,NaN,NaN,...,0.0,3896642.179,3896642.179,0.0,False,True,5.196047e+08,0.235409,5.541738e-02,1210.445136


In [160]:
# TODO: Jion HHI table"
df_hhi_production_egsehi = pd.merge(df_production_egsehi, df_hhi_master, how="left", left_on=["Year", "Country", "Product"], right_on=["Year", "Country", "Product"])
df_hhi_production_egsehi

,Year,Country,Domestic Production Value (P_AC),Product,ISO3,value_economic,no_value_economic,value_governance,no_value_governance,value_social,...,cifvalue,fobvalue,primaryValue,legacyEstimationFlag,isReported,isAggregate,Total_netWgt_YearProduct,Share in % (exports),Share HHI Exports,HHI_exports
0,1970,ALBANIA,NaN,ALUMINIUM,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1971,ALBANIA,NaN,ALUMINIUM,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1972,ALBANIA,NaN,ALUMINIUM,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1973,ALBANIA,NaN,ALUMINIUM,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1974,ALBANIA,NaN,ALUMINIUM,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69784,2016,RUSSIAN FEDERATION,5.584791e+08,PETROLEUM,RUS,0.624176,False,0.358850,False,0.608937,...,NaN,7.371230e+10,7.371230e+10,0.0,False,True,1.508749e+12,16.892819,285.367322,1154.313467
69785,2017,RUSSIAN FEDERATION,5.585467e+08,PETROLEUM,RUS,0.642995,False,0.365093,False,0.582633,...,NaN,9.337726e+10,9.337726e+10,0.0,False,True,9.716810e+11,NaN,NaN,730.396645
69786,2018,RUSSIAN FEDERATION,5.678849e+08,PETROLEUM,RUS,0.645651,False,0.370905,False,0.607808,...,NaN,1.292011e+11,1.292011e+11,0.0,False,True,1.250927e+12,20.829205,433.855770,978.296637
69787,2019,RUSSIAN FEDERATION,5.733888e+08,PETROLEUM,RUS,0.670757,False,0.378865,False,0.599214,...,NaN,1.222286e+11,1.222286e+11,0.0,False,True,1.261201e+12,21.342775,455.514034,1290.653310


In [97]:
# GET entries whose hhi indexes are amepty
# df_hhi_production_egsehi[df_hhi_production_egsehi["HHI_production"].isna() & df_hhi_production_egsehi["HHI_exports"].isna()]

# All the rows have infiormation
# df_hhi_production_egsehi[df_hhi_production_egsehi["HHI_production"].isna()]

# In this case we have a lot of rows with HhI_exports empty because export data contains information from 1970 to 2020 whilst the rest of data is from 2000 to 2020
df_hhi_production_egsehi[df_hhi_production_egsehi["HHI_exports"].isna()  & (df_hhi_production_egsehi["Year"] >= 2000)][["Year", "Country", "Product"]].drop_duplicates().iloc[100:120]

,Year,Country,Product
250,2016,CHINA,ALUMINIUM
251,2017,CHINA,ALUMINIUM
252,2018,CHINA,ALUMINIUM
253,2019,CHINA,ALUMINIUM
254,2020,CHINA,ALUMINIUM
285,2000,CROATIA,ALUMINIUM
286,2001,CROATIA,ALUMINIUM
287,2002,CROATIA,ALUMINIUM
288,2003,CROATIA,ALUMINIUM
289,2004,CROATIA,ALUMINIUM


In [ ]:
df_hhi_master[(df_hhi_master["Year"]==2016) & (df_hhi_master["Country"] == "CHINA")]

In [99]:
df_hhi_master[(df_hhi_master["Year"]==2009) & (df_hhi_master["Country"] == "CROATIA")]

,Year,Country,Value,Product,reporterCode,reporterDesc_x,Total_value_YearProduct,Share in % (production),Share HHI Production,HHI_production,...,cifvalue,fobvalue,primaryValue,legacyEstimationFlag,isReported,isAggregate,Total_netWgt_YearProduct,Share in % (exports),Share HHI Exports,HHI_exports
294,2009,CROATIA,500.0,ALUMINIUM,191.0,CROATIA,197735840.0,0.000253,6.393950e-08,1713.133234,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6720,2009,CROATIA,NaN,BARYTES,191.0,CROATIA,7662051.0,NaN,NaN,2359.851398,...,NaN,9777.0,9777.0,0.0,True,False,3.581624e+09,0.000142,2.027591e-08,3146.684482
47963,2009,CROATIA,NaN,SILVER,191.0,CROATIA,22322463.0,NaN,NaN,981.801726,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [161]:
# FOR NULL DATA BUT FROM 1970 TO 2020
df_hhi_production_egsehi_1970and2020 = pd.merge(df_hhi_master, df_production_egsehi, how="left", left_on=["Year", "Country", "Product"], right_on=["Year", "Country", "Product"])

In [162]:
df_hhi_production_egsehi_1970and2020

,Year,Country,Value,Product,reporterCode,reporterDesc_x,Total_value_YearProduct,Share in % (production),Share HHI Production,HHI_production,...,value_ecosystems,no_value_ecosystems,value_habitat,no_value_habitat,value_infrastructure,no_value_infrastructure,value_governance_isolated,no_value_governance_isolated,EGSEHI,EGSEHI_6root
0,1970,ALBANIA,NaN,ALUMINIUM,8.0,ALBANIA,57280000.0,NaN,NaN,1026.755380,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1971,ALBANIA,NaN,ALUMINIUM,8.0,ALBANIA,60982000.0,NaN,NaN,1132.922206,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1972,ALBANIA,NaN,ALUMINIUM,8.0,ALBANIA,68897000.0,NaN,NaN,1128.647391,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1973,ALBANIA,NaN,ALUMINIUM,8.0,ALBANIA,75087000.0,NaN,NaN,1185.642408,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1974,ALBANIA,NaN,ALUMINIUM,8.0,ALBANIA,83974000.0,NaN,NaN,1223.009237,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87512,2020,NaN,NaN,ZIRCONIUM,757.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87513,2020,NaN,NaN,ZIRCONIUM,784.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87514,2020,NaN,NaN,ZIRCONIUM,792.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87515,2020,NaN,NaN,ZIRCONIUM,826.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Save conformed table

In [144]:
df_hhi_production_egsehi

,Year,Country,Domestic Production Value (P_AC),Product,ISO3,value_economic,no_value_economic,value_governance,no_value_governance,value_social,...,cifvalue,fobvalue,primaryValue,legacyEstimationFlag,isReported,isAggregate,Total_netWgt_YearProduct,Share in % (exports),Share HHI Exports,HHI_exports
0,1970,ALBANIA,NaN,ALUMINIUM,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1971,ALBANIA,NaN,ALUMINIUM,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1972,ALBANIA,NaN,ALUMINIUM,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1973,ALBANIA,NaN,ALUMINIUM,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1974,ALBANIA,NaN,ALUMINIUM,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69784,2016,RUSSIAN FEDERATION,5.584791e+08,PETROLEUM,RUS,0.624176,False,0.358850,False,0.608937,...,NaN,7.371230e+10,7.371230e+10,0.0,False,True,1.508749e+12,16.892819,285.367322,1154.313467
69785,2017,RUSSIAN FEDERATION,5.585467e+08,PETROLEUM,RUS,0.642995,False,0.365093,False,0.582633,...,NaN,9.337726e+10,9.337726e+10,0.0,False,True,9.716810e+11,NaN,NaN,730.396645
69786,2018,RUSSIAN FEDERATION,5.678849e+08,PETROLEUM,RUS,0.645651,False,0.370905,False,0.607808,...,NaN,1.292011e+11,1.292011e+11,0.0,False,True,1.250927e+12,20.829205,433.855770,978.296637
69787,2019,RUSSIAN FEDERATION,5.733888e+08,PETROLEUM,RUS,0.670757,False,0.378865,False,0.599214,...,NaN,1.222286e+11,1.222286e+11,0.0,False,True,1.261201e+12,21.342775,455.514034,1290.653310


In [163]:
df_hhi_production_egsehi.to_csv("../data/hhi_production_exports_egsehi_conformed_table.csv", index=False)

In [164]:
df_hhi_production_egsehi_1970and2020.to_csv("../data/hhi_production_exports_egsehi_null_19702020_conformed_table.csv", index=False)